In [53]:
from pathlib import Path
import polars as ps

# Data Preprocessing

In [54]:
type_map: list = [ps.Utf8, ps.Utf8, ps.Float32, ps.Utf8, ps.Utf8,
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32,
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32,
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32,
                  ps.Float32, ps.Datetime, ps.Float32, ps.Float32, ps.Float32, ps.Int32]
null_values: list = ['', '-']
datasets: list = [ps.read_csv(file, encoding='utf8', dtypes=type_map, null_values=null_values, n_threads=8)
                  for file in Path('data').glob('*.csv')]

In [55]:
dataset: ps.DataFrame = ps.concat(datasets)
dataset = dataset.unique(maintain_order=True)

In [56]:
dataset.head()

SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,PM2.5,PM2.5_AVG,NO2,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId
str,str,f32,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,datetime[μs],f32,f32,f32,i32
"""新莊""","""新北市""",48.0,null,"""良好""",0.7,0.62,0.6,2.6,6.9,12.0,15.0,8.0,7.0,29.200001,43.099998,13.8,1.0,13.0,2022-06-01 00:00:00,0.0,121.432503,25.037971,7
"""菜寮""","""新北市""",47.0,null,"""良好""",0.8,0.67,0.7,0.9,5.3,11.0,10.0,8.0,8.0,28.799999,36.599998,7.7,1.5,145.0,2022-06-01 00:00:00,0.0,121.481026,25.068951,8
"""平鎮""","""桃園市""",27.0,null,"""良好""",0.8,0.33,0.2,6.3,12.7,18.0,16.0,8.0,7.0,16.4,17.200001,0.7,0.2,147.0,2022-06-01 00:00:00,0.0,121.203987,24.952785,20
"""中山""","""臺北市""",37.0,null,"""良好""",0.5,0.5,0.7,3.0,4.2,12.0,11.0,1.0,4.0,22.799999,25.700001,2.8,0.4,186.0,2022-06-01 00:00:00,0.0,121.526527,25.062361,12
"""基隆""","""基隆市""",23.0,null,"""良好""",0.6,0.3,0.2,2.1,7.1,21.0,16.0,4.0,3.0,14.4,17.1,2.7,0.3,330.0,2022-06-01 00:00:00,1.0,121.760056,25.129168,1
